## Machine Learning Operations (MLOps)

In [12]:
from IPython.core.display import HTML

def load_css():
    styles = open("css/custom.css", "r").read()
    return HTML(f"<style>{styles}</style>")

load_css()

## Importando os pacotes do projeto
Vamos centralizar nesta célula a importação de todos os pacotes que iremos utilizar neste notebook

In [ ]:
from pathlib import Path
import sys 
import os

current_path = Path(os.getcwd())
parent_path = current_path.parent.absolute()
sys.path.append(str(parent_path))

import json 
import requests
from dotenv import load_dotenv
from mlflow.models import infer_signature
from mlflow import MlflowClient
import mlflow

import warnings
warnings.filterwarnings('ignore')

<div class="custom-slide">
    <div class="hands-on">
        Continuando nossa implementação
    </div>
</div>

In [14]:
# Carregar variáveis do arquivo .env
load_dotenv()

# Configurar serviço do MLFlow
mlflow.set_tracking_uri(os.getenv('MLFLOW_TRACKING_URI'))

## Usando o modelo em modo batch

Podemos carregar e usar nosso modelo para fazer predições em modo batch, sem a necessidade de um endpoint para inferência.

Primeiramente, vamos criar uma função para retornar as informações do modelo do MLFlow

In [15]:
def get_model_uri_by_name_and_alias(model_name, alias):
    
    # Initialize MLflow client
    client = mlflow.tracking.MlflowClient()
    
    # Get the model version details by alias
    model_version_details = client.get_model_version_by_alias(name=model_name, alias=alias)
    
    # Construct the model URI
    model_uri = f"models:/{model_name}/{model_version_details.version}"

    return model_uri, model_version_details.run_id, model_version_details.version

Podemos recuperar as informações do nosso modelo da seguinte forma

In [16]:
# Example usage
model_name = 'prod.mlops.kmeans-clustering'
alias = 'champion'
model_uri, run_id, version = get_model_uri_by_name_and_alias(model_name, alias)

Já com as informações de referência do modelo, podemos carregá-lo

In [17]:
# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(model_uri)
#logged_model = f"runs:/{run_id}/model"

# Access the underlying custom model
custom_model_instance = loaded_model._model_impl.python_model

# Predict on a Pandas DataFrame.
#import pandas as pd
#loaded_model.predict(pd.DataFrame(data))

E usar o modelo

In [18]:
data = {
    'method': 'predict',
    'data': [-15.65246980577859, -47.7609277270516]
} 
loaded_model.predict(data)

{'is_region_covered': True,
 'closest_center': {'id': 4,
  'distance_in_km': 3.26,
  'lat': np.float64(-15.879763880435013),
  'lng': np.float64(-47.49671343775075)}}

In [9]:
data = {
    'method': 'get_cluster_centroids',
    'data': None
} 
loaded_model.predict(data)

[{'lat': np.float64(-15.654655490491816),
  'lng': np.float64(-47.8442808719988),
  'cluster': 1},
 {'lat': np.float64(-15.623268671816016),
  'lng': np.float64(-47.652359043495856),
  'cluster': 2},
 {'lat': np.float64(-15.756974903901021),
  'lng': np.float64(-47.771778958240816),
  'cluster': 3},
 {'lat': np.float64(-15.649905394173633),
  'lng': np.float64(-47.79121659966453),
  'cluster': 4},
 {'lat': np.float64(-15.879763880435013),
  'lng': np.float64(-47.49671343775075),
  'cluster': 5}]

In [11]:
data = {
    'method': 'get_model_version',
    'data': None
} 
loaded_model.predict(data)

{'version': '2',
 'run_id': 'c4f3fd9aa45240309c4bec233f485937',
 'model_uri': 'models:/prod.mlops.kmeans-clustering/2'}

## Servindo o modelo via API

https://mlflow.org/docs/latest/deployment/deploy-model-locally.html

Basta acessar o terminal e digitar 

```
export MLFLOW_TRACKING_URI=http://localhost:5000
export AWS_ACCESS_KEY_ID=user
export AWS_SECRET_ACCESS_KEY=password
export MLFLOW_S3_ENDPOINT_URL=http://localhost:9000
mlflow models serve -m "models:/prod.mlops.kmeans-clustering@champion" -p8001  --no-conda
```

In [20]:
payload = json.dumps(
    {
        'inputs': {
            'method': 'get_model_version',
            'data': None
        }
    }
)

response = requests.post(
    url=f"http://localhost:8001/invocations",
    data=payload,
    headers={"Content-Type": "application/json"},
)
print(response.json())

{'predictions': {'version': '2', 'run_id': 'c4f3fd9aa45240309c4bec233f485937', 'model_uri': 'models:/prod.mlops.kmeans-clustering/2'}}


In [21]:
payload = json.dumps(
    {
        'inputs': {
            'method': 'get_cluster_centroids',
            'data': None
        }
    }
)

response = requests.post(
    url=f"http://localhost:8001/invocations",
    data=payload,
    headers={"Content-Type": "application/json"},
)
print(response.json())

{'predictions': [{'lat': -15.654655490491816, 'lng': -47.8442808719988, 'cluster': 1}, {'lat': -15.623268671816016, 'lng': -47.652359043495856, 'cluster': 2}, {'lat': -15.756974903901021, 'lng': -47.771778958240816, 'cluster': 3}, {'lat': -15.649905394173633, 'lng': -47.79121659966453, 'cluster': 4}, {'lat': -15.879763880435013, 'lng': -47.49671343775075, 'cluster': 5}]}


In [15]:
payload = json.dumps(
    {
        'inputs': {
            'method': 'predict',
            'data': [-15.913988770795076, -47.60220527168726]
        }
    }
)

response = requests.post(
    url=f"http://localhost:8001/invocations",
    data=payload,
    headers={"Content-Type": "application/json"},
)
print(response.json())

{'predictions': {'is_region_covered': False, 'closest_center': {'id': 4, 'distance_in_km': 11.53, 'lat': -15.603417375334981, 'lng': -47.917044479085696}}}


<div class="custom-slide">
    <h1>Conclusão</h1>
    <p>
       Parabéns por ter acompanhado o curso até aqui! Realmente fizemos muitas coisas interessantes:
    </p>
    <ul>
        <li>Aulas 1, 2 e 3: Exploramos o problema e criamos um modelo completo de ML</li>
        <li>Aula 4: Analisamos o erro do modelo e registramos os resultados para futura análise de model drift</li>
        <li>Aula 5: Aprendemos como utilizar um model registry (MLFlow)</li>
        <li>Aula 6: Aprendemos como criar um modelo customizado, incorporando algoritmos de ML e regras de negócio. </li>
        <li>Aula 7: Aprendemos como implementar os pipelines de inferência (batch e online)</li>
    </ul>
    <p>
       Nas próximas aulas, iremos sair um pouco do ambiente do jupyter notebook para realizar as seguintes tarefas:
    </p>
    <ul>
        <li>Automatizar o pipeline de (re)treino do modelo</li>
        <li>Implementar um processo de CI/CD para promover nosso modelo para o ambiente de produção</li>
        <li>Implementar nossa própria API para servir o modelo</li>
        <li>Analisar o model drift</li>
    </ul>
    <p>
       Vamos em frente para a parte final do curso 🚀
    </p>
</div>

<div class="custom-slide">
<h1>Discutindo sobre alta escalabilidade</h1>
    <img src="images/mlflow-local-serving.png" alt="Sample Image" style="width:100%;border-radius:10px;">
    <span class="image-ref">Source: https://mlflow.org/docs/latest/deployment/deploy-model-locally.html</span>
</div>

<div class="custom-slide">
    <h1>Arquitetura de referência</h1>
    <img src="images/mlflow-prod-deployment.png" alt="Sample Image" style="width:100%;border-radius:10px;">
    <span class="image-ref">Source: https://mlflow.org/docs/latest/deployment/index.html#id1</span>
</div>